In [1]:
# Notebook 02: Generación e Ingesta de Datos
# **Universidad:** Universidad Nacional Experimental de Guayana (UNEG)  
# **Asignatura:** Sistemas de Bases de Datos II  
# **Proyecto:** Proyecto N2 - Data Pipeline Escalable
# ---
# **Descripción:**  
# Genera dataset sintético y realiza la ingesta masiva en Apache Cassandra.

In [2]:
# Instalar dependencia necesaria (solo necesario la primera vez)
!pip install cassandra-driver

In [3]:
import random
import uuid
import time
import json
import os
from datetime import datetime, timedelta
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement, SimpleStatement
from cassandra import ConsistencyLevel
from decimal import Decimal

In [4]:
# Configuración de conexión
# NOTA: Usamos 'cassandra' como host porque estamos dentro de la red de Docker.
CASSANDRA_HOST = ['cassandra'] 
KEYSPACE = 'ventas_db'
TABLE = 'ventas_crudas'
NUM_REGISTROS = 100000
BATCH_SIZE = 100

# Archivo de métricas compartido (directorio docs montado por Docker)
METRICS_FILE = '../docs/metricas.json'

In [5]:
def setup_database():
    print("Conectando a Cassandra...")
    cluster = Cluster(CASSANDRA_HOST)
    session = cluster.connect()
    
    print(f"Creando Keyspace '{KEYSPACE}' si no existe...")
    session.execute(f"""
        CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
        WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': 1}};
    """)
    
    session.set_keyspace(KEYSPACE)
    
    # ⚠️ DROP TABLE para forzar recreación con esquema correcto
    print(f"🗑️ Eliminando tabla '{TABLE}' si existe (para recrear con esquema correcto)...")
    session.execute(f"DROP TABLE IF EXISTS {TABLE}")
    
    print(f"✨ Creando Tabla '{TABLE}' con esquema completo...")
    session.execute(f"""
        CREATE TABLE {TABLE} (
            id_venta UUID,
            id_cliente UUID,
            id_producto UUID,
            fecha_venta DATE,
            categoria TEXT,
            producto TEXT,
            cantidad INT,
            precio_unitario DECIMAL,
            monto_total DECIMAL,
            metodo_pago TEXT,
            region TEXT,
            PRIMARY KEY (fecha_venta, id_venta)
        ) WITH CLUSTERING ORDER BY (id_venta ASC);
    """)

    return cluster, session

cluster, session = setup_database()
print("✅ Base de datos configurada correctamente.")

Conectando a Cassandra...
Creando Keyspace 'ventas_db' si no existe...
🗑️ Eliminando tabla 'ventas_crudas' si existe (para recrear con esquema correcto)...
✨ Creando Tabla 'ventas_crudas' con esquema completo...
✅ Base de datos configurada correctamente.


In [6]:
# Lista expandida a 15 categorías para hacer significativo el 'Top 10'
CATEGORIAS = [
    'Electronica', 'Ropa', 'Hogar', 'Alimentos', 'Deportes',
    'Juguetes', 'Libros', 'Salud', 'Automotriz', 'Jardin',
    'Mascotas', 'Belleza', 'Musica', 'Cine', 'Videojuegos'
]
METODOS_PAGO = ['Tarjeta Credito', 'Tarjeta Debito', 'Efectivo', 'Transferencia']
REGIONES = ['Norte', 'Sur', 'Este', 'Oeste', 'Centro']

def generar_registro():
    fecha = datetime(2024, 1, 1) + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    cantidad = random.randint(1, 10)
    precio = Decimal(str(round(random.uniform(5.0, 150.0), 2)))
    monto = cantidad * precio
    
    return (
        uuid.uuid4(),                          # id_venta
        uuid.uuid4(),                          # id_cliente
        uuid.uuid4(),                          # id_producto
        fecha.date(),                          # fecha_venta
        random.choice(CATEGORIAS),             # categoria
        f"Producto {random.randint(1, 100)}",  # producto
        cantidad,                              # cantidad
        precio,                                # precio_unitario
        monto,                                 # monto_total
        random.choice(METODOS_PAGO),           # metodo_pago
        random.choice(REGIONES)                # region
    )

In [7]:
print(f"--- Iniciando Ingesta de {NUM_REGISTROS} registros ---")
start_time = time.time()

insert_query = session.prepare(f"""
    INSERT INTO {TABLE} (id_venta, id_cliente, id_producto, fecha_venta, categoria, producto, cantidad, precio_unitario, monto_total, metodo_pago, region)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

batch = BatchStatement(consistency_level=ConsistencyLevel.ONE)
count_in_batch = 0
total_inserted = 0

for _ in range(NUM_REGISTROS):
    datos = generar_registro()
    batch.add(insert_query, datos)
    count_in_batch += 1
    
    if count_in_batch >= BATCH_SIZE:
        session.execute(batch)
        batch = BatchStatement(consistency_level=ConsistencyLevel.ONE)
        count_in_batch = 0
        total_inserted += BATCH_SIZE
        
        if total_inserted % 10000 == 0:
            print(f"Insertados: {total_inserted}...")

# Insertar remanentes
if count_in_batch > 0:
    session.execute(batch)
    total_inserted += count_in_batch

end_time = time.time()
duration_ingesta = end_time - start_time

print(f"✅ Ingesta Completada.")
print(f"Total registros: {total_inserted}")
print(f"Tiempo total: {duration_ingesta:.2f} segundos")
print(f"Tasa: {total_inserted / duration_ingesta:.2f} regs/seg")

--- Iniciando Ingesta de 100000 registros ---
Insertados: 10000...
Insertados: 20000...
Insertados: 30000...
Insertados: 40000...
Insertados: 50000...
Insertados: 60000...
Insertados: 70000...
Insertados: 80000...
Insertados: 90000...
Insertados: 100000...
✅ Ingesta Completada.
Total registros: 100000
Tiempo total: 13.30 segundos
Tasa: 7518.02 regs/seg


In [8]:
# Validación Final
row = session.execute(f"SELECT COUNT(*) FROM {TABLE}").one()
print(f"📊 Registros en base de datos: {row[0]}")

cluster.shutdown()

📊 Registros en base de datos: 100000


In [9]:
# =====================================================
# 📊 GUARDAR MÉTRICAS PARA NOTEBOOK 04
# =====================================================

# Leer métricas existentes o crear nuevo diccionario
if os.path.exists(METRICS_FILE):
    with open(METRICS_FILE, 'r') as f:
        metricas = json.load(f)
else:
    metricas = {}

# Actualizar con métricas de este notebook
metricas['ingesta_cassandra'] = {
    'tiempo_segundos': round(duration_ingesta, 2),
    'registros': total_inserted,
    'tasa_regs_seg': round(total_inserted / duration_ingesta, 2),
    'timestamp': datetime.now().isoformat()
}

# Guardar en docs
with open(METRICS_FILE, 'w') as f:
    json.dump(metricas, f, indent=2)

print(f"✅ Métricas guardadas en: {METRICS_FILE}")
print(f"   - Tiempo de ingesta: {duration_ingesta:.2f} segundos")

✅ Métricas guardadas en: ../docs/metricas.json
   - Tiempo de ingesta: 13.30 segundos
